In [ ]:
# Download the data set it is around 160 MB
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
# Loading Dateset
movies_df = pd.read_csv('/home/sakthi/Documents/machineLearning/ml-latest/movies.csv')
ratings_df = pd.read_csv('/home/sakthi/Documents/machineLearning/ml-latest/ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [34]:
# Extract Year from title
movies_df['year']= movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
# Remove the Brackets in the Year column
movies_df['year']=movies_df.year.str.extract('(\d\d\d\d)',expand=False)
# Remove the year in title 
movies_df['title']=movies_df.title.str.replace('(\(\d\d\d\d\))','')
# Remove the spaces in title
movies_df['title']=movies_df['title'].apply(lambda x:x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [10]:
# Split Genres Based on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji (),"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men (),"[Comedy, Romance]",1995
3,4,Waiting to Exhale (),"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II (),[Comedy],1995


In [35]:
# Local copy of Movie Dataframe
moviesWithGenres_df = movies_df.copy()
# Iterate Over each rows on Genres
for index,row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index,genre]=1
# Fill Zeros in place of Not Available values
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,A,d,v,e,n,t,...,S,-,I,X,W,N,(,,g,)
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,Adventure|Children|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,Comedy|Romance,1995,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,Comedy,1995,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Check the Rating Data set
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [15]:
# Remove the Timestamp columns
ratings_df=ratings_df.drop('timestamp',1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [39]:
# Getting User Input From user
userInput =[
    {'title':'Breakfast Club, The','rating':5},
    {'title':'Toy Story', 'rating':3.5},
    {'title':'Jumanji','rating':2},
    {'title':'Pulp Fiction','rating':5},
    {'title':'Akira','rating':4.5}
]
# Convert the data into dataframe
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [40]:
# Getting the list (Movie Id, Title, Genre, Year) of user inputted Movies
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
# Merge 
inputMovies =pd.merge(inputId, inputMovies)
# Remove the genre and year columns
inputMovies = inputMovies.drop('genres',1).drop('year',1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [41]:
# Find the UserMovie id with genre rating
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,A,d,v,e,n,t,...,S,-,I,X,W,N,(,,g,)
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,Adventure|Children|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,Comedy|Crime|Drama|Thriller,1994,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,Action|Adventure|Animation|Sci-Fi,1988,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The",Comedy|Drama,1985,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Reset the Index 
userMovies = userMovies.reset_index(drop = True)
# Remove the unwanted Columns
userGenreTable = userMovies.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)
userGenreTable

,A,d,v,e,n,t,u,r,|,i,...,S,-,I,X,W,N,(,,g,)
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# User Rating
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [44]:
# Element wise multiplication of 
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

A    10.0
d    20.0
v    10.0
e    20.0
n    10.0
t    10.0
u    10.0
r    20.0
|    20.0
i    15.0
m    18.0
a    20.0
o    18.0
C    15.5
h    10.5
l    10.5
y    15.5
F    10.0
s     5.5
R     0.0
c     4.5
D    10.0
T     5.0
H     0.0
M     0.0
S     4.5
-     4.5
I     0.0
X     0.0
W     0.0
N     0.0
(     0.0
      0.0
g     0.0
)     0.0
dtype: float64

In [45]:
# Set Index of MovieId
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
genreTable = genreTable.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)
genreTable.head()

,A,d,v,e,n,t,u,r,|,i,...,S,-,I,X,W,N,(,,g,)
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
genreTable.shape

(34208, 35)

In [47]:
# Recommendation for 
recommendationTable_df = ((genreTable * userProfile) .sum(axis = 1)) / (userProfile.sum())
recommendationTable_df.head()

movieId
1    0.904040
2    0.782828
3    0.543771
4    0.644781
5    0.360269
dtype: float64

In [48]:
# Sort the value
recommendationTable_df = recommendationTable_df.sort_values(ascending = False)
recommendationTable_df.head()

movieId
6902      1.000000
5018      1.000000
81132     0.984848
148775    0.983165
144350    0.983165
dtype: float64

In [49]:
# Find the Corresponding Movie for each movie id
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
542,546,Super Mario Bros.,Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,1993
664,673,Space Jam,Adventure|Animation|Children|Comedy|Fantasy|Sc...,1996
4923,5018,Motorama,Adventure|Comedy|Crime|Drama|Fantasy|Mystery|S...,1991
6252,6350,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,Action|Adventure|Animation|Children|Fantasy|Sc...,1986
6793,6902,Interstate 60,Adventure|Comedy|Drama|Fantasy|Mystery|Sci-Fi|...,2002
8605,26093,"Wonderful World of the Brothers Grimm, The",Adventure|Animation|Children|Comedy|Drama|Fant...,1962
8942,26590,G.I. Joe: The Movie,Action|Adventure|Animation|Children|Fantasy|Sc...,1987
9218,27155,"Batman/Superman Movie, The",Action|Adventure|Animation|Children|Fantasy|Sc...,1998
9825,32031,Robots,Adventure|Animation|Children|Comedy|Fantasy|Sc...,2005
12021,54278,Underdog,Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,2007
